In [ ]:
import typing
import weave
from weave import weaveflow
import models
import example_adaptor

In [ ]:
weave.init('self-reflection3')

In [ ]:
dataset_ref = weaveflow.publish_huggingface_dataset("commonsense_qa", None, "validation", 10, 0)

In [ ]:
@weave.type()
class CommonsenseQAAdaptor(example_adaptor.ExampleAdaptor):
    @weave.op()
    def example_to_prompt(self, example: typing.Any) -> str:
        labels = example["choices"]["label"]
        texts = example["choices"]["text"]
        answers = "\n".join(f"{label}: {text}" for label, text in zip(labels, texts))
        return f"""\n{example['question']}\n\n{answers}\n\n"""

adaptor = CommonsenseQAAdaptor("Just provide a single letter (A-E) as your response. Respond with a single character and nothing else!")

In [ ]:
chat_model = weaveflow.OpenaiChatModel('gpt-3.5-turbo')
single_step_model = models.SingleStepModel(chat_model, adaptor)
reflect_model = models.SelfReflectModel(chat_model, adaptor)

In [ ]:
@weave.op()
def example_answer(example: typing.Any) -> str:
    return example['answerKey']

evaluation_single = weaveflow.EvaluateNamedSteps(
    ['final_answer'],
    weaveflow.EvaluateExactMatch(example_answer))

evaluation_reflect = weaveflow.EvaluateNamedSteps(
    ['first_answer', 'final_answer'],
    weaveflow.EvaluateExactMatch(example_answer))

In [ ]:
weaveflow.evaluate(evaluation_single, dataset_ref, single_step_model)
weaveflow.evaluate(evaluation_reflect, dataset_ref, reflect_model)